V tomto notebooku načteme naše data uložená v csv souboru.  


### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`

In [1]:
# Prikaz ktery naistaluje knihovny
%pip install pandas 
%pip install pymarc 

# Prikaz ktery knihovny prida 
from collections import Counter
import pandas as pd


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


### 1. Načtení z csv

Nejprve pomocí knihovny pandas načteme naše uložená csv data do datové struktury DataFrame (která je podobná např. excelovské tabulce). Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jmeno autora).
Některá pole a podpole mohou opakovat, ty jsou  v csv spojené středníkem. V DataFramu pak funkcí `split()` hodnoty rozpojíme a převede do listu (seznamu). Pokud zrovna na daném místě žádná hodnota není, přidáme prázdný list.


In [12]:
# Cesta k nasim datum
csv_data = 'data/csv/out_smz.csv'

# Nacteni dat
df = pd.read_csv(csv_data, delimiter=',')

print("Data načtena do DataFramu df.")

for column in df.columns:
    # Rok vydani bude vzdy jen jedenkrat
    if column != 'year': 
        # Hodnoty spojene v jeden string zpatky rozdelime do listu, aby se nam s nim lepe pracovalo
        df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])
              

Data načtena do DataFramu df.


### 2. Souhrnné statistiky

Nyní se podíváme, jak z dat zjistit jednoduché statistiky. 

#### 2.1 Počet výskytů 

Pokud chceme vědet, kolikrát se v databázi objevuje nějaká hodnota, použijeme funkci `value_counts()`. V následujícím příkladu si ukážeme, kolik je v databázi z jakého roku záznamů.<br>
Pomocí parametru `records_to_print` omezíme počet vypsaných záznamů na prvních x. Hodnoty se vypíší sestupně. 

In [13]:
records_to_print = 20

df['year'].value_counts()[:records_to_print] # vypise prvnich x zaznamu

year
1989    2428
1988    2299
1987    2105
1986    1519
1984    1352
1983    1348
1985    1252
1979    1232
1980    1169
1981    1109
1982     943
1977     832
1978     786
1976     477
198u     375
1990     284
1972     264
1975     256
1974     173
1991     166
Name: count, dtype: int64

Vidíme, že zdaleka nejvíce záznamů je z 80. let, následně pak z let 70. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Místo sloupce 'year' můžeme vypsat např. 'author', čímž zjistíme, kolikrát se jaký autor v databázi objevuje.<br>
</div>

#### 2.2 Nejčastější autor v daném roce

Teď si ukážeme, jak z dat zjistit, který autor v jakém roce se v záznamech v daném roce nejčastěji objevuje jako autor. Existuje několik způsobů, jak na to přijít. My si představíme dva.<br>

##### 2.2.1 První varianta - vlastní funkce

Nejprve si napíšeme dvě vlastní funkce.<br> 
První funkce `count_years(df, column)` nám spočte počet článků každého autora v každém roce. Jako vstupy přijímá DataFrame `df` a vybraný sloupec `column`. Výsledky pak vrátí jako dictionary, kde klíčem je tuple (rok, jméno autora) a hodnota je počet článků.<br>
Druhá funkce `find_most_common(count)` pak pro každý rok zjistí, který autor v každém roce napsal nejvíce článků. Vstupem je dictionary `count`, který nám vrátí funkce `count_years(df, column)`. Funkce pak postupně projde všechny klíče dictionary `count` a pro každý rok si pamatuje autora s nejvyšším počtem záznamů. Funkce nakonec vrátí setřízený dictionary, kde klíč je rok a hodnota je tuple (jmeno autora, počet záznamů).   


In [10]:
# Funkce ktera nam spocte pocet zaznamu v danem roce pro kazdeho autora
def count_years(df, column):
    
    # Dictionary, ve kterem jako klic ulozime (rok, jmeno autora)
    # a jako hodnotu budeme pripocitavat pocet clanku
    count = {}
    
    # Projdeme radky DataFramu
    for _,row in df.iterrows():
        year = row['year']
        for element in row[column]:
            key = (year, element) 
            # Pokud klic jiz existuje, pripocteme jednicku
            if key in count.keys():
                count[key] += 1
            else:
                # Pokud klic zatim neexistuje, vytvorime ho a pridame jedna
                count[key] = 1  
    return count              

# Funkce, ktera nam najde nejcastejsiho autora v danem roce
def find_most_common(count):
    # Dictionary, ve kterem jako klic ulozime rok
    # a jako hodnotu (jmeno autora, pocet clanku)
    most_common = {}

    # Projdeme klice a hodnoty dictionary
    for key,value in count.items():
        year = key[0]
        # Pokud klic (rok) jiz existuje, podivame se, 
        # zda je pocet clanku u aktualniho autora vyssi  
        if year in most_common:
            if most_common[year][1] < value:
                # Pamatujeme si jen tu nejvyssi hodnotu
                most_common[year] = (key[1], value)    
        else:
            most_common[year] = (key[1], value) 
    
    #Klice slovniku seradime od nejmensiho cisla po nejvetsi         
    years = list(most_common.keys())
    years.sort()
    sorted_most_common = {i: most_common[i] for i in years}        
    return sorted_most_common            

print("Funkce uloženy.")
  

Funkce uloženy.


Teď už jen funkce zavoláme a výsledek vypíšeme. <br> 
Abychom měli hezčí výpis, použijeme formátování. Předem si řekneme, v jakém formátu budeme chtít hodnoty vypsat a jak velké mezi nimi budou mezery. 

In [12]:
# Vybereme sloupec, ktery chceme vypsat
picked_column = 'author'

# Zavolame nase funkce
count = count_years(df, picked_column)

most_common = find_most_common(count)

# Formatovani pro hezci vypis
string = "{year: >25} {name: >25} {articles: >25}"
# Nejcastejsi hodnoty vypiseme
print(string.format(year = "Rok", name = "Nejčastější autor", articles = "Počet záznamů"))
print("----------------------------------------------------------------------------------------------------")
for key, value in most_common.items():
    print(string.format(year = key, name = value[0], articles = value[1]))        


                      Rok         Nejčastější autor              Počet článků
----------------------------------------------------------------------------------------------------
                     1951    Effenberger, Vratislav                        11
                     1953             Havel, Václav                         6
                     1954                   Kopecký                         2
                     1958    Effenberger, Vratislav                         6
                     1960    Effenberger, Vratislav                        13
                     1962    Effenberger, Vratislav                         2
                     1966             Budík, Arnošt                         6
                     1967             Budík, Arnošt                         7
                     1968             Budík, Arnošt                         3
                     1969             Budík, Arnošt                         2
                     196u              Te

V tabulce vidíme, že počet napsaných článků za jednotlivé roky se výrazně liší. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Kód je napsán tak, aby se parametry daly jednoduše upravit. Můžeme si tak zkusit změnit proměnnou picked_colum na 'figures', čímž zjistíme, o kom nejvíce se za daný rok psalo.
</div>

#### 2.2.2 Druhá varianta - lambda a Counter

Tím, že máme data uložena do DataFramu, umožňuje nám to využít funkce knihovny pandas. V našem případě využijeme funkci `groupby()`, která data seskupí podle vybraného sloupce. <br>
Použijeme také funkci `Counter()`, která nám spočte výskyt každého elementu. V neposlední řadě využijeme funkci `lambda`.<br>

In [14]:
# Data ocistime o prazdne hodnoty
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Data seskupime podle nejcastejsiho elementu v kazde hodnote groupby
most_common = data_filtered.groupby('year')[picked_column].apply(lambda x: Counter(element for elements in x for element in elements).most_common(1)[0][0] if len(x) > 0 else None) 

print(most_common)

year
1951               Effenberger, Vratislav
1953                        Havel, Václav
1954                              Kopecký
1958               Effenberger, Vratislav
1960               Effenberger, Vratislav
1962               Effenberger, Vratislav
1966                        Budík, Arnošt
1967                        Budík, Arnošt
1968                        Budík, Arnošt
1969                        Budík, Arnošt
196u                         Teige, Karel
1970                      Novotný, Michal
1971                      Holan, Vladimír
1972                       Fibich, Ondřej
1973                       Fibich, Ondřej
1974                      Řezníček, Pavel
1975                         Daumal, René
1976                      Pavlovský, Petr
1977                      Pavlovský, Petr
1978                      Pavlovský, Petr
1979                        Vrabec, Milan
197u                    Nezval, Vítězslav
1980                         Štogr, Josef
1981                         

V tabulce můžeme vidět, který autor za daný rok publikoval nejvíce článků napříč všemi časopisy. Je třeba si uvědomit, že i když je v tabulce nějaký autor v daný rok nejčastější, nemusí v celkovém počtu napsat hodně článků/knih. Ostatně se o tom můžeme přesvěsvědčit v předchozí buňce.<br>
Je možné, že se od sebe dvě tabulky budou mírně odlišovat a to v případě, že databáze za daný rok obsahuje více autorů se stejným počtem záznamů. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Kromě bibliografie literárního exilu můžeme vyzkoušet i jiné bibliografie. Stačí si v předchozím Jupyter notebooku vytvořit csv tabulku. 
</div>
